In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from bioinfokit import analys, visuz
import os 
import glob
from tqdm import tqdm
from src.config import intersection

### Define directories

In [ ]:
# Define directories
Base='/home/projects/cpr_10006/people/lilniu/projects/target/data/data_genotype/subset/Formatted/plink_qc4/gemma/'
result_path = Base + 'results/'
figure_path = Base + 'analysis/figures/'
annotation_path = Base + 'analysis/annotation/'
crossref_path = Base + 'analysis/cross_reference'

### Read summary statistics

In [ ]:
# Read summary statistics after filtering for genome-wide significance (<5x10-8)
sig_folder = 'sig/'
os.chdir(os.path.join(result_path,sig_folder))
my_files = glob.glob('*.txt')
files = []
for i in tqdm(my_files):
    df = pd.read_csv(os.path.join(result_path,sig_folder,i), sep='\t').drop(['Unnamed: 0'], axis=1)
    files.append(df)
    
# Check if any file is missing
print('Numer of files: {}'.format(len(my_files)))
missing_files = [i for i in np.arange(1,421) if i not in sorted([int(i.split('_')[0]) for i in my_files])]
print('Missing file:'.format(missing_files))

In [ ]:
# Combine all significant hits
df_sig = pd.concat(files)

# Check if any reference and alternative SNPs are flipped
df_sig['REF'] = df_sig['rs'].str.split('_').str[2]
df_sig['ALT'] = df_sig['rs'].str.split('_').str[3]
flipped = df_sig[df_sig['REF']!= df_sig['allele0']]
print('Reference and alternative SNPs were flipped in the following rows: {}'.format(flipped))

# Create two new columns of 'Protein ID' and 'Gene name' by splitting 'phenotype'
df_sig['Protein ID'] = df_sig['phenotype'].str.split('_').str[0]
df_sig['Gene name'] = df_sig['phenotype'].str.split('_').str[1]

In [ ]:
# Check inflation factor lambda
# script: https://github.com/pgxcentre/lambda/blob/master/compute_lambda.py
lambda_log = pd.read_csv(os.path.join(result_path,'assoc/log.lambda.txt'), sep=' ',header=None, )
lambda_log[3].describe()

### Export variants to be annotated by variant effect predictor

In [ ]:
# Prepare data for VEP annotation
target5_bim = pd.read_csv(os.path.join(Base,'target5.bim'), sep='\s+', header=None, 
            names=['CHR', 'SNP', 'dummy', 'POS', 'ALT', 'REF'])
target5_bim['allele']=target5_bim['REF']+'/'+target5_bim['ALT']
target5_bim['strand']='+'
target5_bim_sig = target5_bim[target5_bim['SNP'].isin(df_sig['rs'])]
vep_input = target5_bim_sig[['CHR', 'POS', 'POS', 'allele','strand', 'SNP']]
vep_input.to_csv(os.path.join(Base, 'target5.vep.input'), sep='\t', header=None, index=False)

In [ ]:
# https://grch37.ensembl.org/Homo_sapiens/Tools/VEP with RefSeq transcripts as reference
# Read VEP annotation
vep_output = pd.read_csv(os.path.join(Base, 'target5.vep.output.txt'), sep='\t', na_values='-', low_memory=False)

# Create a dictionary that matches SNP to rsID
rsid = vep_output[['#Uploaded_variation', 'Existing_variation']].drop_duplicates()
IDmapping_snp_to_rs = dict(zip(rsid['#Uploaded_variation'], rsid['Existing_variation'].str.split(',').str[0]))

### Primary pQTLs

#### Read clump results

- physical distance (±1 Mb) and LD threshold of r2 > 0.2, significance level 5e-8
- check script 
/home/projects/cpr_10006/people/lilniu/projects/target/data/data_genotype/subset/Formatted/plink_qc4/gemma/analysis/script_clump.sh

In [ ]:
# Create a dictionary that matches phenotype ID and Protein ID
phenotype_to_proteinID = pd.read_csv(os.path.join(Base,'ProteinID.txt'), sep='\t')
IDmapping_phenotype_to_proteinID = dict(zip(phenotype_to_proteinID['Phenotype ID'], phenotype_to_proteinID['Protein ID']))

In [ ]:
# Combine clumped results for all proteins
clump_path = result_path + 'assoc/clump_results/'
os.chdir(clump_path)
clump_files = glob.glob('*.clumped')

files = [pd.read_csv(file, sep='\s+') for file in clump_files]

for df, file in zip(files, clump_files):
    phenotype = int(file.split('.')[0])
    proteinID = IDmapping_phenotype_to_proteinID[phenotype]
    df['phenotype'] = proteinID
    
clumped_snps = pd.concat(files, ignore_index=True)

#### Clumping of the HLA region

In [ ]:
# exclude HLA region (chr6: 29691116–33054976)
df = clumped_snps.copy()
hla_region = (df.CHR == 6) & (df.BP > 29691116) & (df.BP < 33054976)
clumped_hlaremoved = df[~hla_region]

# identify the index of the SNP with the smallest p-value for each protein in the HLA region
df_hla = df[hla_region]
to_keep = df_hla.groupby('phenotype')['P'].idxmin()

# filter out the SNPs from the HLA region and keep the selected SNPs with the smallest p-value
clumped_refined = pd.concat([clumped_hlaremoved, df.query('index in @to_keep')])

In [ ]:
# Compute the number of significant associations, proteins, and SNPs
gw_sig_threshold = 5e-8
study_wide_sig_threshold = gw_sig_threshold / 420

for threshold_type, threshold in [('genome-wide', gw_sig_threshold), ('study-wide', study_wide_sig_threshold)]:
    significant = clumped_refined['P'] < threshold
    nr_associations = significant.sum()
    nr_proteins = clumped_refined.loc[significant, 'phenotype'].nunique()
    nr_snps = clumped_refined.loc[significant, 'SNP'].nunique()
    print(f"At {threshold_type} significance threshold,\nafter clump and HLA region removed,\n{nr_associations} primary associations were identified,\ninvolving {nr_proteins} proteins, and \n{nr_snps} SNPs\n")

In [ ]:
# Find the ids for primary associations
in_both = intersection(df_sig.set_index(['rs', 'phenotype']).index, clumped_refined.set_index(['SNP', 'phenotype']).index)
df_sig_primary = df_sig.set_index(['rs','phenotype']).loc[in_both].reset_index()

# Compute additional columns
df_sig_primary = df_sig_primary.assign(maf=[a if a <0.5 else 1-a for a in df_sig_primary['af']])
df_sig_primary = df_sig_primary.assign(beta_abs=abs(df_sig_primary['beta']))
df_sig_primary = df_sig_primary.assign(maf_log10=np.log10(df_sig_primary.maf))
df_sig_primary = df_sig_primary.assign(beta_abs_log10=[np.log10(a) for a in df_sig_primary['beta_abs']])
df_sig_primary['rsID']=df_sig_primary['rs'].map(IDmapping_snp_to_rs)

# Save data
df_sig_primary.to_csv(os.path.join(Base,'analysis/dataset/df_sig_primary.csv'), index=False)

In [ ]:
for i in df_sig_primary[df_sig_primary['phenotype']=='P22891_PROZ']['rsID']:
    print(i)

#### Assess post-clump pair-wise LD between primary pQTLs of the same protein

In [ ]:
# Import LD results
# plink --file target5_primary_out --r2 --ld-window 99999 --ld-window-kb 99999 --ld-window-r2 0
file = '/home/projects/cpr_10006/people/lilniu/projects/target/data/data_genotype/subset/Formatted/plink_qc4/plink.ld'
df_ld = pd.read_csv(file, sep='\s+')

nr_snps = len(set(df_ld['SNP_A'].unique().tolist() + df_ld['SNP_B'].unique().tolist()))
print('Number of SNPs pair-wise LD calculation: {}'.format(nr_snps))

In [ ]:
# Group the DataFrame by phenotype and extract the rs column as a Series
grouped = df_sig_primary.groupby('phenotype')['rs']

# Create a dictionary comprehension that maps each phenotype to a list of rs values
rs_dict = {phenotype: list(rs_values) for phenotype, rs_values in grouped}

In [ ]:
# Extract pairwise LD results for each protein
ld_primary = []
for phenotype in df_sig_primary['phenotype'].unique():
    pqtls = rs_dict[phenotype]
    mask = df_ld['SNP_A'].isin(pqtls) & df_ld['SNP_B'].isin(pqtls)
    df = df_ld.loc[mask].copy()
    df['phenotype'] = phenotype
    ld_primary.append(df)

# Concatenate pairwise results from all proteins 
df_ld_postclump = pd.concat(ld_primary)

In [ ]:
# Extract pairwise LD with r2 <=0.2
le = df_ld_postclump[df_ld_postclump['R2']<=0.2]
le_rate = pd.DataFrame(le.groupby('phenotype')['R2'].count()/df_ld_postclump.groupby('phenotype')['R2'].count())
le_rate.columns=['%_R2<0.2']
le_rate_mean = le_rate['%_R2<0.2'].mean()
print('On average, {}% of pair-wise LD have a R2 <=0.2'.format(round(le_rate_mean, 2)*100))

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
df_ld_postclump['R2'].hist(bins=50)
plt.xlabel('LD (r2)')

#### Annotate primary pQTLs with VEP

In [ ]:
# Extract SNPs
primary_pqtls = df_sig_primary['rs'].unique()
primary_pqtls_vep = vep_output[vep_output['#Uploaded_variation'].isin(primary_pqtls)]
consequences = primary_pqtls_vep[['#Uploaded_variation', 'Consequence']].drop_duplicates()

In [ ]:
# Create a dictionary that matches SNPs and variant annotation
dict_vep = {}
for i, group in consequences.groupby('#Uploaded_variation'):
    dict_vep[i] = group['Consequence'].tolist()

#### Extract missense variants to eliminate artefactual pQTLs

In [ ]:
# Check how many missense variants are in significant pQTLs before and after clumping
missense_variants_all = vep_output.loc[vep_output['Consequence'] == 'missense_variant', '#Uploaded_variation']
missense_variants_primary = consequences.loc[consequences['Consequence'] == 'missense_variant', '#Uploaded_variation']
print(f"{len(missense_variants_all)} missense variants in all significant pQTLs")
print(f"{len(missense_variants_primary)} missense variants in primary pQTLs")

In [ ]:
# Filter and select relevant columns from vep_output
coding_cons = vep_output.loc[vep_output['Consequence'] == 'missense_variant', ['#Uploaded_variation', 'SYMBOL', 'Protein_position', 'Amino_acids']].drop_duplicates()

# Extract SNP to affected protein mapping
IDmapping_snp_to_affectedprot = coding_cons[['#Uploaded_variation', 'SYMBOL']].drop_duplicates().set_index('#Uploaded_variation')['SYMBOL'].to_dict()

# Map SNP IDs to affected proteins in df_sig
df_sig_coding_cons = df_sig.assign(affected_protein=df_sig['rs'].map(IDmapping_snp_to_affectedprot))

# Select only the rows where the affected protein matches the gene name
df_sig_coding_tocheck = df_sig_coding_cons.loc[df_sig_coding_cons['Gene name'] == df_sig_coding_cons['affected_protein'], :].sort_values('phenotype')

# Save file to path
df_sig_coding_tocheck.to_csv(os.path.join(Base, 'analysis/alphamap/df_sig_coding_tocheck.csv'), index=False)

# Extract coding consequences of all missense variants and save file to path
coding_cons_allsig = coding_cons[coding_cons['#Uploaded_variation'].isin(df_sig_coding_tocheck['rs'].unique())]
coding_cons_allsig.to_csv(os.path.join(Base,'analysis/alphamap/coding_cons_allsig.csv'), index=False)

#### Plot figures

##### Figure 4i

In [ ]:
from mycolorpy import colorlist as mcp

# Compute values for the pie chart
df_pie = consequences['Consequence'].value_counts() / consequences['#Uploaded_variation'].nunique()
df_pie['others'] = df_pie[df_pie < 0.008].sum()
toplot_pie = (df_pie[df_pie > 0.006] * 100).round().astype(int)

# Generate labels for the pie chart
labels = [f"{index}: {value}%" for index, value in toplot_pie.items()]

# Generate colors for the pie chart
colors = mcp.gen_color(cmap="Paired", n=toplot_pie.shape[0])

# Plot the pie chart
fig, ax = plt.subplots()
ax.pie(toplot_pie, colors=colors, wedgeprops={"edgecolor":"black", 'linewidth': 1, 'linestyle': 'solid', 'antialiased': True})
ax.legend(labels, loc=(1, 0.1), labelcolor='black')
plt.rcParams['pdf.fonttype'] = 42
fig.savefig(os.path.join(figure_path, 'vep_pie.pdf'), dpi=120, bbox_inches='tight')

In [ ]:
# Compute the number of proteins per SNP and SNPs per protein
protein_per_snp = df_sig_primary.groupby('rs')['Protein ID'].count().value_counts()
snp_per_protein = df_sig_primary.groupby('Protein ID')['rs'].count().value_counts()

# Compute the number of chromosomes per protein and identify the proteins with more than one chromosome
chr_per_protein = df_sig_primary.groupby('Protein ID')['chr'].nunique()
# Sort proteins by the number of chromosomes
chr_per_protein = chr_per_protein.sort_values(ascending=False).to_frame(name='chr')
proteins_with_multiple_chromosomes = chr_per_protein[chr_per_protein > 1].shape[0]

# Prepare data for plotting
snp_per_protein_plot = snp_per_protein[snp_per_protein.index.isin(range(1, 11))].sort_index()
snp_per_protein_plot['>10'] = snp_per_protein[snp_per_protein.index > 10].sum()

##### Figure 4f-g

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6,3))

# Plot number of associated proteins per SNP
axs[0].bar(x=protein_per_snp.index, height=protein_per_snp, 
        facecolor='#53BBD5', edgecolor='#3B5488', width=0.6)
axs[0].set_xticks(ticks=np.arange(1,7), labels=np.arange(1,7), rotation=45);
axs[0].set_xlabel('Number of associated\nproteins per SNP', fontsize=12)
axs[0].set_ylabel('Number of pQTLs', fontsize=12)
axs[1].bar(x=np.arange(11), height=snp_per_protein_plot, 
          facecolor='#53BBD5', edgecolor='#3B5488', width=0.6)

# Plot number of associated SNPs per protein
axs[1].set_xticks(ticks=np.arange(11), labels=[i for i in np.arange(1, 11)] + ['>10'], rotation=45);
axs[1].set_xlabel('Number of associated\n SNPs per protein', fontsize=12)
axs[1].set_ylabel('Number of pQTLs', fontsize=12)
plt.subplots_adjust(wspace=0.35)
plt.rcParams['pdf.fonttype'] = 42
plt.savefig(os.path.join(figure_path, 'protein_snp_frequency.pdf'), dpi=120, bbox_inches='tight')

### cis/trans pQTLs

In [ ]:
# Import mapping file from Protein ID to Transcription start site - refer to notebook TSS.ipynb
df_mapping = pd.read_csv(os.path.join(Base, 'analysis/annotation/TSS_mapping_primary_pQTLs.csv'))

# Add columns of protein coding location
df_sig2 = df_sig_primary.merge(df_mapping, how='left', on='Protein ID').drop_duplicates()
df_sig2 = df_sig2.assign(pqtl_id=df_sig2['rs']+df_sig2['phenotype']).set_index('pqtl_id')

# Categorize cis and trans pQTLs 
from src.config import cis_trans_pqtl
df_sig2_cistrans = cis_trans_pqtl(data=df_sig2, 
                                  chr_snp='chr', 
                                  pos_snp='ps', 
                                  chr_prot='chr_protein', 
                                  pos_protein ='Transcription start site (TSS)')

df_sig2_cistrans['values'].value_counts()

##### Fig. 4d

In [ ]:
cis_trans_prop =df_sig2_cistrans['values'].value_counts(1, dropna=False).round(2)*100
cis_trans_prop = cis_trans_prop.rename({'NaN':'Unmapped', 'cis':'Cis', 'trans':'Trans'})
cis_trans_nr = df_sig2_cistrans['values'].value_counts(dropna=False)
labels=[cis_trans_prop.index[i] +  ": {} ({}%)".format(cis_trans_nr.iloc[i], int(cis_trans_prop.iloc[i])) for i in np.arange(cis_trans_prop.shape[0])]

fig, ax = plt.subplots(figsize=(2,2))
colors=['white', '#53BBD5', 'gray']
plt.pie(cis_trans_prop, colors=colors,
        wedgeprops={"edgecolor":"black",'linewidth': 1, 'linestyle': 'solid', 'antialiased': True});
plt.legend(labels, loc=(-0.2,-0.5), labelcolor='black', fontsize=12)
plt.rcParams['pdf.fonttype'] = 42
plt.savefig(os.path.join(figure_path, 'cis_trans_pie.pdf'), dpi=120, bbox_inches='tight')

#### how many proteins have cis, trans only and cis/trans associations

In [ ]:
cistranscount = df_sig2_cistrans.groupby(['phenotype'])['values'].value_counts().unstack().sort_values(by=['cis'], ascending=False)

In [ ]:
cis_only = cistranscount[cistranscount['cis'].notnull() & cistranscount['trans'].isnull()]
trans_only = cistranscount[cistranscount['trans'].notnull() & cistranscount['cis'].isnull()]
both = cistranscount[(cistranscount['cis'].notnull())&(cistranscount['trans'].notnull())]
ctcount = pd.Series({'cis_only': len(cis_only), 'trans_only': len(trans_only), 'cis/trans': len(both)})

##### Fig. 4e

In [ ]:
fig, ax=plt.subplots(figsize=(3,3))
plots=plt.bar(x=ctcount.index, height=ctcount, width=0.4, facecolor='#53BBD5', edgecolor='#3B5488')
plt.rcParams['pdf.fonttype'] = 42
plt.ylabel('Number of proteins', fontsize=12)
for bar in plots.patches:
    plt.annotate(bar.get_height(), xy=(bar.get_x() + bar.get_width()/2, bar.get_height()), 
                ha='center', va='center', xytext=(0, -9), textcoords='offset points',
                 color='white', fontsize=12)
plt.xticks(ticks=[0,1,2], labels=['Cis only', 'Trans only', 'Cis/Trans']);
plt.rcParams['pdf.fonttype'] = 42
plt.savefig(os.path.join(figure_path, 'cis_trans_bar.pdf'), dpi=120, bbox_inches='tight')

In [ ]:
df_sig2_cistrans['-log10(p-wald)'] = [-np.log10(i) for i in df_sig2['p_wald']]
df_sig2_cistrans['ind']=range(len(df_sig2_cistrans))
df_sig2_cistrans = df_sig2_cistrans.sort_values(by='chr')

In [ ]:
df_sig3=df_sig2_cistrans.copy()
df_sig3['rsID']=df_sig3['rs'].map(IDmapping_snp_to_rs)
df_sig3=df_sig3.reset_index()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_plot = df_sig3.copy()
df_plot.chr = df_plot.chr.astype('category')
df_plot_grouped = df_plot.groupby('chr')

##### Fig. 4a

In [ ]:
# plot all primary pQTLs across the 22 chromosomes
fig, ax = plt.subplots(figsize=(6, 3))
sig_thres = -np.log10(5e-8)

colors=['#3B5488','#53BBD5']
x_labels = []
x_labels_pos = []
x_labels_end = []
for num, (name, group) in enumerate(df_plot_grouped):
    group.plot(kind='scatter', x='ind', y='-log10(p-wald)',color=colors[num % len(colors)], ax=ax, s=1, )
    x_labels.append(name)
    x_labels_pos.append((group['ind'].iloc[-1] - (group['ind'].iloc[-1] - group['ind'].iloc[0])/2))
    x_labels_end.append(group['ind'].iloc[-1])
major_tick_pos = [x_labels_pos[i] for i in [4, 9, 14, 19]]
ax.set_xticks(major_tick_pos, labels=[5, 10, 15, 20], rotation=90);

ax.set_xticks(x_labels_pos, minor=True)
plt.axhline(y=sig_thres, color='gray', linestyle='--')
ax.set_xlabel('pQTL position (Chromosome)', fontsize=14)
plt.ylabel('$-log_{10}{(P)}$', fontsize=14)
plt.grid(which='minor', ls =':');

plt.rcParams['pdf.fonttype'] = 42
plt.savefig(os.path.join(figure_path, 'manhattan_sig_sizeclear.pdf'), dpi=120, bbox_inches='tight')

##### Figure 4b

In [ ]:
fig, ax = plt.subplots(figsize=(6,1))
nr_bychr = df_plot_grouped.count()[['pqtl_id']]
plt.bar(x=nr_bychr.index, height=nr_bychr['pqtl_id'], 
        facecolor='#53BBD5', edgecolor='#3B5488', width=0.6)
plt.xticks(np.arange(1, 23), labels=[i for i in np.arange(1, 23)]);
plt.ylabel('Number\nof pQTLs', fontsize=12)
plt.xlabel('Chromosome', fontsize=12)
plt.rcParams['pdf.fonttype'] = 42
plt.savefig(os.path.join(figure_path, 'pqtls_by_chr.pdf'), dpi=120, bbox_inches='tight')

##### Figure 4c

In [ ]:
df_toplot = df_sig3.copy().dropna()

In [ ]:
# Read chromosome length
df_chr_len = pd.read_csv(os.path.join(annotation_path, 'GRCh37_genome_length.txt'), sep='\t', header=None, names=['chr', 'total_length_bp', 'Genbank_accession', 'Refseq_accession'])
# Remove sex chromosomes
df_chr_len = df_chr_len[~df_chr_len['chr'].isin(['X', 'Y'])].astype({'chr': int})

cum_chr_length = np.cumsum(df_chr_len.set_index('chr')['total_length_bp'])
cum_chr_length.loc[0] = 0

xticks = [(cum_chr_length[i] - cum_chr_length[i - 1]) / 2 + cum_chr_length[i - 1] for i in range(1, 23)]

major_ticks = [xticks[i] for i in [4, 9, 14, 19]]


In [ ]:
new_pos_snp = [cum_chr_length[i-1] for i in df_toplot['chr']] + df_toplot['ps'] 
new_pos_prot = [cum_chr_length[int(i)-1] for i in df_toplot['chr_protein']] + df_toplot['Transcription start site (TSS)'] 
df_toplot = df_toplot.assign(ps_snp = new_pos_snp, ps_prot = new_pos_prot )

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
ax =sns.scatterplot(x='ps_snp', y='ps_prot', data=df_toplot, hue='values', 
                palette = {'cis':'red', 'trans':'#3B5488'}, s=8, legend=None)
plt.xticks(major_ticks, labels=[5,10,15,20]);
plt.yticks(major_ticks, labels=[5,10,15,20]);

ax.set_xticks(xticks, major=True)
ax.set_yticks(xticks, major=True)
ax.set_xticks(cum_chr_length, minor=True)
ax.set_yticks(cum_chr_length, minor=True)
plt.xlabel('pQTL position', fontsize=14);
plt.ylabel('Protein coding gene position', fontsize=14);
plt.xlim(-1e8, cum_chr_length[22]+1e8)
plt.ylim(-1e8, cum_chr_length[22]+1e8)
plt.grid(which='minor', ls =':');

plt.rcParams['pdf.fonttype'] = 42
plt.savefig(os.path.join(figure_path, 'cis_vs_trans.pdf'), dpi=120, bbox_inches='tight')

### Replication in ALD study

In [ ]:
# Import SNPs in the ALD study
ald_vcf_path = '/home/projects/cpr_10006/people/lilniu/projects/proteogenomics/data/data_genotype/pqtl/data_genotype_subset/plink_qc2/'
path_ald = ald_vcf_path + 'gemma/analysis/'
bim_ald = ald_vcf_path + 'gemma/ald5.bim'
snps_ald = pd.read_csv(bim_ald, header=None, sep='\t', names=['chr', 'rs', 'what', 'pos', 'allele1', 'allele0'])
snps_ald['rs'] = snps_ald['rs'].str.replace(':', "_")

# Import proteins in the ALD study
proteins_ald = pd.read_csv(os.path.join(crossref_path, 'ald/ald_protein.list'))

# Check how many protein-SNP pairs can be tested in the ALD study
cond1 = df_sig_primary['Protein ID'].isin(proteins_ald['Protein ID'].tolist())
cond2 = df_sig_primary['rs'].isin(snps_ald['rs'])

df_to_rep = df_sig_primary[cond1 & cond2]
df_to_rep = df_to_rep.assign(pqtl_id=df_to_rep['rs'] + df_to_rep['Protein ID'], chr=df_to_rep['chr'].astype(str))
df_to_rep.to_csv(os.path.join(path_ald, 'target_to_replicate.csv'), index=False)
print('{} variant-protein pairs need to be tested'.format(df_to_rep.shape[0]))

In [ ]:
prot_chr_to_rep = pd.DataFrame(df_to_rep.groupby('Protein ID')['chr'].unique())
prot_chr_to_rep_dict = dict(zip(prot_chr_to_rep.index, prot_chr_to_rep['chr']))

import pickle
with open(os.path.join(path_ald, 'prot_chr_to_rep.pkl'), 'wb') as handle:
    pickle.dump(prot_chr_to_rep_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Significant pQTLs in the replication cohort(p<0.05) and assign new columns
df_sig_ald = pd.read_csv(os.path.join(crossref_path, 'ald/pqtl_ald_sig_nominal.csv'))
df_sig_ald['Protein ID'] = df_sig_ald['phenotype'].str.split('_').str[0]
df_sig_ald['pqtl_id'] = df_sig_ald['rs'] + df_sig_ald['Protein ID']
df_sig_ald['chr'] = df_sig_ald['chr'].astype(str)
df_sig_ald['maf'] = np.where(df_sig_ald['af'] < 0.5, df_sig_ald['af'].round(3), 0.5)
df_sig_ald['-log10(p-wald)'] = (-np.log10(df_sig_ald['p_wald'])).round(3)
df_sig_ald[['se_3digit', 'beta_3digit']] = df_sig_ald[['se', 'beta']].round(3)

# significant pQTLs after Bonferroni correction
# df_sig_ald = pd.read_csv(os.path.join(crossref_path, 'ald/pqtl_ald_sig.csv'))

#### Replication
- criteria: a pQTL is considered replicated if the SNP or its proxy in LD (+/1Mb, r2 > 0.2) is also sig. associated with the corresponding protein (p < 0.05)

In [ ]:
from src.config import annotate_replication
df_rep = annotate_replication(df_to_rep, df_sig_ald)

In [ ]:
no_replicated = df_rep[df_rep['replicated in ALD'].isin(['vicinity', 'exact'])].shape[0]
per_replicated = no_replicated/df_rep.shape[0]*100
print('{:.0f}% of testable primary pQTLs in the TARGET cohort was replicated in the ALD study.'.format(per_replicated))

In [ ]:
df_rep.to_csv(os.path.join(crossref_path, 'ald/replicated_in_ald.csv'), sep='\t')

#### Check correlation in beta statistics between discovery and replication cohort

In [ ]:
df_exact = df_rep[df_rep['replicated in ALD']=='exact'][['pqtl_id', 'beta', 'rs', 'phenotype']].merge(df_sig_ald[['pqtl_id', 'beta']], 
                                                                        on='pqtl_id', how='left')
df_exact=df_exact.rename({'beta_x':'beta_target', 'beta_y':'beta_ald'}, axis=1)
df_exact['beta_target']=np.float64(df_exact['beta_target'])
df_exact = df_exact.sort_values(by='beta_target')
df_exact['pair_id']=np.arange(df_exact.shape[0])

In [ ]:
df_rep['replicated in ALD'].value_counts()

In [ ]:
df_exact_long = df_exact.melt(id_vars=['pqtl_id', 'pair_id'], value_name='beta', var_name='cohort')
df_exact_long['cohort']=df_exact_long['cohort'].map({'beta_target':'Holbæk', 'beta_ald':'GALA-ALD'})
df_exact[['beta_target', 'beta_ald']].corr()

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
b = sns.scatterplot(x='beta_target', y='beta_ald', data=df_exact, edgecolor='darkblue', color='white')
plt.xlim(-1.55, 1.55)
plt.ylim(-1.55, 1.55);
plt.xlabel('BETA (Holbæk)', fontsize=12)
plt.ylabel('BETA (GALA-ALD)', fontsize=12)
b.tick_params(labelsize=12)
b.set_xticks([-1.5, -1.0, -0.5, 0, 0.5, 1, 1.5])
plt.rcParams['pdf.fonttype'] = 42
#plt.savefig(os.path.join(figure_path, 'beta_concordance_nominal.pdf'), dpi=120, bbox_inches='tight')

##### Fig. 6b

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
a = sns.scatterplot(x='pair_id', y='beta', hue='cohort', 
                data=df_exact_long, 
                    palette={'Holbæk':'darkblue', 'GALA-ALD':'darkred'},alpha=0.3, s=40)
a.tick_params(labelsize=12)
a.set_xlabel('')
a.set_ylabel('BETA', fontsize=12)
a.legend(prop={'size': 12})
a.invert_yaxis()
plt.rcParams['pdf.fonttype'] = 42
plt.savefig(os.path.join(figure_path, 'beta_concordance_dotplot_nominal.pdf'), dpi=120, bbox_inches='tight')

#### Prepare Supplementary Table 9

In [ ]:
df_table9 = df_rep.sort_values(by='replicated in ALD', ignore_index=True)
df_table9[df_table9['replicated in ALD']=='exact'].to_csv(os.path.join(Base, 'analysis/dataset/dataset_replicated.csv'), index=False)
df_table9['maf']=[i if i<=0.5 else 1-i for i in df_table9['af']]
df_table9['-log10(p-wald)']=[-np.log10(i) for i in df_table9['p_wald']]

cols_tokeep =['rs', 'chr', 'REF', 'ALT', 'Protein ID','Gene name', 'maf', 'beta', 'se', 'p_wald', 
             'replicated in ALD', 'evidence', 'maf_replication cohort', 'beta_replication cohort', 
             'se_replication cohort', 'p_wald_replication cohort', '-log10(p-wald)_replication cohort']
df_table9=df_table9[cols_tokeep]

#### Export pQTLs for plotting dose-responses

In [ ]:
with open (os.path.join(ald_vcf_path, 'exact_match_pqtl_id.txt'), 'w') as f:
    for line in df_exact['rs'].tolist():
        f.write(line.replace('_', ':') + '\n')
df_exact.to_csv(os.path.join(ald_vcf_path, 'exact_match_pqtl.csv'), index=False)

### Cross reference other studies

In [ ]:
# Import prior studies
prior_studies=pd.read_csv(os.path.join(crossref_path, 'all_studies_tidy.txt'), low_memory=False)
prior_studies.dropna(subset=['GeneSymbol', 'Chr.SNP.hg19', 'Pos.SNP.hg19'], inplace=True)
prior_studies.reset_index(drop=True, inplace=True)
prior_studies=prior_studies.astype({'Pos.SNP.hg19':int,
                                   'Chr.SNP.hg19':str})

# Save results
prior_studies.to_csv(os.path.join(crossref_path, 'prior_studies_pqtl_list.txt'), index=False)

In [ ]:
from src.config import estimate_novelty, diff
df_sig4 = estimate_novelty(df_sig3, prior_studies)

In [ ]:
# Select novel significant genes
df_sig4_novel = df_sig4[df_sig4['replicated in nr.studies'] == 0]

# Identify unique genes
unique_genes = sorted(set(df_sig4_novel['Gene name']) - set(prior_studies['GeneSymbol']))

# Print summary
print('{} new genes found compared to existing studies.'.format(len(unique_genes)))


In [ ]:
import ast
import itertools

studies = [i for i in df_sig4['novel'] if i!='novel']
study_occurences = []
for i in studies:
    res = ast.literal_eval(i)
    study_occurences.append(res)

study_occurences = list(itertools.chain(*study_occurences))
study_occurences_df = pd.DataFrame({'occurences':study_occurences})['occurences'].value_counts()/1116

In [ ]:
# Count number of studies in which each gene is replicated
replication_matrix = df_sig4['replicated in nr.studies'].value_counts().sort_index()

replication_matrix_toplot = replication_matrix[replication_matrix.index<11]
replication_matrix_toplot['>10'] = replication_matrix[replication_matrix.index>10].sum()

##### Fig. S4b

In [ ]:
fig, ax= plt.subplots(figsize=(4,3))
plt.bar(x=np.arange(replication_matrix_toplot.__len__()), 
        height=replication_matrix_toplot, width=0.6)
plt.xticks(np.arange(replication_matrix_toplot.__len__()), 
           labels=replication_matrix_toplot.index, 
          rotation=30);
plt.ylabel('Number of pQTLs', fontsize=14)
plt.xlabel('Number of studies', fontsize=14)
plt.rcParams['pdf.fonttype'] = 42
plt.savefig(os.path.join(figure_path, 'replication_matrix.pdf'), dpi=120, bbox_inches='tight')

#### Write to results

In [ ]:
vcf_path ='/home/projects/cpr_10006/people/lilniu/projects/target/data/data_genotype/subset/Formatted/plink_qc4'
with open (os.path.join(vcf_path, 'novel_pqtl_id.txt'), 'w') as f:
    for line in df_sig4_novel['rs'].tolist():
        f.write(line + '\n')

df_sig4_novel.to_csv(os.path.join(vcf_path, 'novel_pqtl.csv'), index=False)

with open (os.path.join(vcf_path, 'primary_pqtl_id.txt'), 'w') as f:
    for line in df_sig2['rs'].tolist():
        f.write(line + '\n')

df_sig4.to_csv(os.path.join(vcf_path, 'primary_pqtl.csv'), index=False)

#### How many of the novel pQTLs were replicated in ALD? 

In [ ]:
# Extract pQTLs replicated in the replication cohort
df_replicated_ald = df_rep[df_rep['replicated in ALD']!='no']

# Extract the relevant columns from df_replicated_ald and df_sig4_novel
replicated_ids = df_replicated_ald['rs']+df_replicated_ald['Protein ID']
novel_ids = df_sig4_novel['rs']+df_sig4_novel['Protein ID']

# Compute the intersection of the two sets and count the number of elements
num_common_ids = len(intersection(replicated_ids, novel_ids))

# Print the result
print('The number of common IDs between the two datasets is: {}'.format(num_common_ids))

#### Prepare Supplementary Table 5

In [ ]:
df_table5 = df_sig4.copy()
df_table5 = df_table5.reset_index().drop(['index', 'ind'], axis=1)
df_table5['study-wide significant']=np.where(df_table5['p_wald']<5e-8/420, 'yes', 'no')
df_table5['variant annotation'] = df_table5['rs'].map(dict_vep)
df_table5['novel'] = df_table5['novel'].replace('[]', 'novel')

cols_tokeep = ['pqtl_id', 'rs', 'rsID',  'chr', 'ps', 'variant annotation', 'REF', 'ALT', 'maf', 
               'p_wald', '-log10(p-wald)', 'beta', 'se', 'study-wide significant', 
              'values', 'Protein ID', 'Gene name', 'UniParc ID','chr_protein',
              'Gene start (bp)', 'Gene end (bp)', 'Transcription start site (TSS)', 
               'novel', 'replicated in nr.studies']

new_names = ['pQTL_ID', 'SNP', 'rsID', 'chr', 'pos', 'VEP annotation','reference_allele', 'alternative_allele', 
            'MAF', 'p_wald', '-log10(p-wald)', 'beta', 'standard error', 'study-wide significant', 
            'cis/trans', 'Protein ID', 'Gene name', 'UniParc ID','chr_protein', 'Gene start (bp)', 
             'Gene end (bp)', 'Transcription start site (TSS)', 'novel', 'replicated in nr.studies']

df_table5_formatted = df_table5[cols_tokeep].rename(dict(zip(cols_tokeep, new_names)), axis=1)
df_table5_formatted['artefactual pQTLs'] = np.where(df_table5_formatted['Protein ID']=='A0A0D9SG88', 'yes', 'no')

### Mapping pQTLs to GWAS Catalogue

In [ ]:
gwas_catal = pd.read_csv(os.path.join(crossref_path, 'gwas_catalog_v1.0.2-associations_e107_r2022-07-30.tsv'), 
                         sep='\t', low_memory=False)

In [ ]:
# Write all mapped GWAS traits for inspection
with open(os.path.join(crossref_path, 'gwas_traits'), 'w') as file:
    for i in gwas_catal['MAPPED_TRAIT'].unique():
        file.write(str(i)+'\n')

In [ ]:
from src.config import annotate_with_gwas

In [ ]:
df_sig4_gwas = annotate_with_gwas(df_sig4, gwas_catal)

#### Prepare Supplementary Table 8

In [ ]:
cols_tokeep = ['pqtl_id', 'rsID', 'Protein ID', 'Gene name','Nr. GWAS_records','replicated in nr.studies',
                'GWAS_traits', 'Nr. GWAS_traits', 'Study accessions', 'Mapped trait urls']

df_table8 = df_sig4_gwas[cols_tokeep]
df_table8 = df_table8[df_table8['Nr. GWAS_traits']>0].rename({'pqtl_id':'pQTL_ID'}, axis=1)
df_table8['novel pQTL'] = np.where(df_table8['replicated in nr.studies']>0, 'no', 'yes')
df_table8=df_table8.drop(['replicated in nr.studies'], axis=1).sort_values(by='Nr. GWAS_traits', ascending=False)

In [ ]:
# Write table 5,8,9 to supplementary tables
with pd.ExcelWriter(os.path.join(Base,'analysis/tables/TableS589.xlsx')) as writer:
    df_table5_formatted.to_excel(writer,sheet_name='ST5', index=False)
    df_table8.to_excel(writer, sheet_name='ST8', index=False)
    df_table9.to_excel(writer, sheet_name='ST9', index=False)
    
df_table5_formatted.to_csv(os.path.join(Base, 'analysis/tables/TableS5.csv'), index=False)

#### Plot regional association plots between GWAS traits loci and pQTLs

In [ ]:
IDmapping_proteinID_to_phenotype = {key:value for (value,key) in zip(IDmapping_phenotype_to_proteinID.keys(), IDmapping_phenotype_to_proteinID.values())}
IDmapping_pqtlid_to_cistrans = dict(zip(df_sig2_cistrans.index, df_sig2_cistrans['values']))

In [ ]:
sig_thres_gwas= 5e-8
sig_thres_study= 1.2e-10

In [ ]:
def plot_regional_association(rsid, proteinid, ax, figureid):
    phenotype = IDmapping_proteinID_to_phenotype[proteinid]
    markers = {"yes": "v", "no": "o"}
    #annotation df
    df = pd.read_csv(os.path.join(result_path, "assoc/{}.assoc.txt".format(phenotype)), sep='\t')
    df['rsID']=df['rs'].map(IDmapping_snp_to_rs)
    chr_pos = df[df['rsID']==rsid].iloc[0]['chr']
    snp_pos = df[df['rsID']==rsid].iloc[0]['ps']
    df = df[(df['chr']==chr_pos) & (df['ps'] > snp_pos-5e5) & (df['ps'] < snp_pos+5e5)]
    df['phenotype']=proteinid
    df['-log10(p-wald)'] = -np.log10(df.p_wald)
    df['Gene name']=df['phenotype'].str.split('_').str[1]
    df['pqtl_id']=df['rs']+df['phenotype']
    df['cis_trans']=df['pqtl_id'].map(IDmapping_pqtlid_to_cistrans)
    df['target_qtl']='no'
    df.loc[df['rsID']==rsid, 'target_qtl']='yes'

    #plot
    ax.text(-0.2, 1.05, figureid, size=12, weight='bold',transform=ax.transAxes )
    sns.scatterplot(x='ps', y='-log10(p-wald)', data=df, 
                    legend=False, s=6,edgecolor='darkblue', 
                    style='target_qtl', color='darkblue', hue='target_qtl', ax=ax)
    ax.axhline(y=-np.log10(sig_thres_gwas), color='gray', linestyle='--', lw=1)
    ax.axhline(y=-np.log10(sig_thres_study), color='darkred', linestyle='--', lw=1)
    ax.set_xlabel('Position on Chromosome {}'.format(chr_pos))
    ax.set_ylabel('$-log_{10}{(P)}$')
    pqtl_info = df[df['ps']==snp_pos]
    if pqtl_info.shape[0]==0:
        print('No target info found')
    else:
        text = '{} ({})'.format(pqtl_info.iloc[0]['Gene name'], pqtl_info.iloc[0]['cis_trans'])
        ax.annotate(text, (0.02, 0.9), xycoords='axes fraction')
        ax.annotate(rsid, (snp_pos+1e4, pqtl_info.iloc[0]['-log10(p-wald)']))
        ax.set_ylim(-0.1,df['-log10(p-wald)'].max()*1.3 )

##### Fig. 5i-k

In [ ]:
fig, axs = plt.subplots(3,1,figsize=(3,8))
fig.subplots_adjust(hspace=0.5, wspace=0.3)
plot_regional_association(rsid='rs173539', proteinid='P02647_APOA1', ax=axs[0], figureid='i')
plot_regional_association(rsid='rs73001065', proteinid='P04114_APOB', ax=axs[1], figureid='k')
plot_regional_association(rsid='rs6762719', proteinid='E7EQB2_LTF', ax=axs[2], figureid='j')

# Save figure
plt.rcParams['pdf.fonttype'] = 42
fig.savefig(os.path.join(figure_path, 'regional_associations.pdf'), dpi=120, bbox_inches='tight')